In [1]:
import os
import arcpy
from arcpy import env
from arcpy.sa import *
from arcpy.da import *
import pandas as pd

In [2]:
def zonstatis(in_table, fields, value_raster,out_table):
    """统计分区矢量内的栅格信息
    :param in_table: 矢量分区
    :param fields: 矢量字段
    :param value_raster: 栅格数据
    :param out_table: 输出文件"""
    ZonalStatisticsAsTable(in_zone_data=in_table,zone_field=fields,in_value_raster=value_raster,out_table=out_table,ignore_nodata='#',statistics_type='ALL')

In [8]:
# 矢量数据路径
polygon_path = r"C:\Users\Runker\Desktop\train_data\train_data.shp"
# 栅格数据路径
raster_path = r'D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb'

In [4]:
# 统计结果表格存放路径
temp_table_gdb_path = r'F:\cache_data\zone_ana\dy\zone_temp.gdb'

# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(temp_table_gdb_path)
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(os.path.dirname(temp_table_gdb_path), "zone_temp", "CURRENT")

In [5]:
# 切换工作空间
env.workspace = raster_path

In [6]:
# 获取需要处理的栅格列表
raster_list = arcpy.ListRasters()

In [9]:
[_.name for _ in arcpy.ListFields(polygon_path)]

['FID',
 'Shape',
 'Join_Count',
 'TARGET_FID',
 'OBJECTID',
 'DLMC',
 'new_TL',
 'new_YL',
 'new_TS',
 'new_TZ',
 'Shape_Leng',
 'Shape_Area',
 'XMin',
 'YMin',
 'XMax',
 'YMax',
 'Centroid_X',
 'Centroid_Y']

In [ ]:

# 逐一统计分析
for one_raster in raster_list:
    print(one_raster)
    # 分区统计
    zonstatis(polygon_path,'OBJECTID_1',one_raster,os.path.join(temp_table_gdb_path,one_raster))

### 构造训练表


In [11]:
# 切换工作空间
env.workspace = temp_table_gdb_path

In [12]:
# 获取表格列表
table_list = arcpy.ListTables()

In [13]:
# 指定需要的信息
line_fields = 'OBJECTID_1'
df_list = []
for one_table in table_list:
    # 将表转换为pandas数据帧
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(one_table, "*"))  # 确保数据表中无空值
    # 提取统计字段的数据
    merged_df = df[[line_fields]+ list(df.columns[-10:])]
    # 在统计数据列名中添加表名
    merged_df.columns = [line_fields]+[one_table + '_' + col for col in merged_df.columns[-10:]]
    # 添加到列表
    df_list.append(merged_df)
# 初始合并数据帧
df_merged = df_list[0]
# 逐个合并其余的数据帧
for df in df_list[1:]:
    df_merged = pd.merge(df_merged, df, on=line_fields, how='outer')

In [14]:
df_merged

,OBJECTID_1,DEM_COUNT,DEM_AREA,DEM_MIN,DEM_MAX,DEM_RANGE,DEM_MEAN,DEM_STD,DEM_SUM,DEM_MEDIAN,...,MRVBF_COUNT,MRVBF_AREA,MRVBF_MIN,MRVBF_MAX,MRVBF_RANGE,MRVBF_MEAN,MRVBF_STD,MRVBF_SUM,MRVBF_MEDIAN,MRVBF_PCT90
0,1,45.0,1125.0,882.599976,892.000000,9.400024,887.297778,2.567621,39928.400024,886.799988,...,45.0,1125.0,0.175973,0.927709,0.751735,0.514265,0.181825,23.141913,0.542814,0.732555
1,2,40.0,1000.0,846.799988,856.000000,9.200012,850.875002,2.596131,34035.000061,850.299957,...,40.0,1000.0,0.020420,2.899949,2.879529,0.740872,0.818252,29.634874,0.280020,1.963257
2,3,15.0,375.0,849.599976,850.400024,0.800049,850.079997,0.225688,12751.199951,850.099976,...,15.0,375.0,0.356556,2.441020,2.084464,1.249620,0.717455,18.744295,1.555661,2.078964
3,4,9.0,225.0,848.299988,849.500000,1.200012,848.899997,0.385874,7640.099976,848.900024,...,9.0,225.0,2.318881,2.856664,0.537784,2.654204,0.173236,23.887836,2.721920,2.826278
4,5,70.0,1750.0,798.900024,812.500000,13.599976,808.059996,4.837571,56564.199707,810.449982,...,70.0,1750.0,0.175895,3.922467,3.746572,2.210734,1.132333,154.751347,1.997966,3.688263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32574,32575,51.0,1275.0,839.700012,865.700012,26.000000,852.223533,7.477222,43463.400208,851.000000,...,51.0,1275.0,0.002571,0.210302,0.207732,0.052170,0.057858,2.660665,0.026408,0.130794
32575,32576,36.0,900.0,587.599976,606.700012,19.100037,598.038886,5.037541,21529.399902,598.800018,...,36.0,900.0,0.000769,0.889754,0.888985,0.362129,0.348733,13.036649,0.149137,0.864730
32576,32577,40.0,1000.0,812.000000,818.200012,6.200012,814.272501,1.430036,32570.900024,814.100006,...,40.0,1000.0,0.000007,2.979848,2.979841,1.892070,0.988962,75.682809,1.987226,2.952691
32577,32578,24.0,600.0,974.900024,977.500000,2.599976,975.754166,0.786328,23418.099976,975.400024,...,24.0,600.0,4.839455,4.960456,0.121002,4.910208,0.040304,117.844982,4.896280,4.958958


In [15]:
polygon_path = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features_dy\train_datas'

In [2]:
# 添加字段
def add_field(input_table,field_name,field_type='DOUBLE'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)

In [4]:

# 新增面特征信息
# 添加新字段来存储四至坐标
for one_fields in ['XMin', 'YMin', 'XMax', 'YMax','Centroid_X','Centroid_Y']:
    add_field(polygon_path,one_fields)


In [5]:
# 更新字段值
with arcpy.da.UpdateCursor(polygon_path, ["SHAPE@", "XMin", "YMin", "XMax", "YMax", "Centroid_X", "Centroid_Y"]) as cursor:
    for row in cursor:
        centroid = row[0].centroid
        extent = row[0].extent
        row[1] = extent.XMin
        row[2] = extent.YMin
        row[3] = extent.XMax
        row[4] = extent.YMax
        row[5] = centroid.X
        row[6] = centroid.Y
        cursor.updateRow(row)

print("四至坐标计算完成并存储在新字段中。")


四至坐标计算完成并存储在新字段中。


In [17]:
polygon_fields_list = [_.name for _ in arcpy.ListFields(polygon_path)]

In [18]:
print(polygon_fields_list)

['OBJECTID_1', 'Shape', 'Join_Count', 'TARGET_FID', 'OBJECTID', 'DLMC', 'new_TL', 'new_YL', 'new_TS', 'new_TZ', 'Shape_Length', 'Shape_Area', 'XMin', 'YMin', 'XMax', 'YMax', 'Centroid_X', 'Centroid_Y']


In [20]:
polygon_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(polygon_path, ['OBJECTID_1','DLMC','new_TZ', 'XMin', 'YMin', 'XMax', 'YMax', 'Centroid_X', 'Centroid_Y']))

In [24]:
# 连接表格
result_df = pd.merge(polygon_df, df_merged, on='OBJECTID_1',how='left')

In [26]:
# 保存训练数据
train_data_path = r'F:\cache_data\zone_ana\dy\train_data'
result_df.to_csv(os.path.join(train_data_path, 'result.csv'), index=False)